# Sequential Graphs
## Basic Chatbot 


In [19]:
from typing import TypedDict # Imports all the data types we need
from langgraph.graph import StateGraph, START, END
from langchain_ollama import ChatOllama
from dotenv import load_dotenv
from rich import print
# Load environment variables from .env file
load_dotenv('../.env')



True

In [20]:
llm = ChatOllama(model="llama3.2")
llm.invoke("Hello, world!").content

"Hello! It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [21]:
class AgentState(TypedDict):
    name: str
    age: str
    final: str
    message: str

### Node Fuctions

In [29]:
def greating_message(state: AgentState) -> AgentState:
    """Function to greet the user and initialize the state."""
    print("Welcome to the chatbot!")
    state['name'] = input("What is your name? ")
    state['age'] = input("How old are you? ")
    
    return state

def chatbot(state: AgentState) -> AgentState:
    """Chatbot function that updates the state with user input."""
    user_input = input("You: ")
    state['message'] = llm.invoke(user_input).content
    print(f"Chatbot: {state['message']}")
    return state


In [30]:
# simple_chatbot.py

from typing_extensions import TypedDict
from langchain_core.messages import HumanMessage, AIMessage, AnyMessage
from langgraph.graph import StateGraph, START, END
from langchain.chat_models import ChatOpenAI

# 1) Define the shared state schema
class State(TypedDict):
    name: str
    user_question: str
    messages: list[AnyMessage]

# 2) Node 1: Greet the user by name
def greet(state: State) -> dict:
    greeting = AIMessage(f"Hi {state['name']}!")
    # Append to existing messages
    return {"messages": state.get("messages", []) + [greeting]}

# 3) Node 2: Take the user's question and get an LLM response
def answer(state: State) -> dict:
    # Build the message history: include the user's question
    history = state["messages"] + [HumanMessage(state["user_question"])]
    # Call the chat model
    model = ChatOpenAI(model_name="gpt-3.5-turbo")
    response: AIMessage = model(history)
    return {"messages": history + [response]}

# 4) Assemble the graph
builder = StateGraph(State)
builder.add_node(greet)
builder.add_node(answer)
# Define control flow: START → greet → answer → END
builder.add_edge(START, "greet")
builder.add_edge("greet", "answer")
builder.add_edge("answer", END)
graph = builder.compile()

# 5) Invoke the graph
initial_state = {
    "name": "Alice",
    "user_question": "What's the capital of France?",
    "messages": []
}
result = graph.invoke(initial_state)

# 6) Print out the conversation
for msg in result["messages"]:
    msg.pretty_print()


/var/folders/m7/yjmck8kn59gc9w3kdklj2lt40000gn/T/ipykernel_70971/3948258616.py:25: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI(model_name="gpt-3.5-turbo")
/var/folders/m7/yjmck8kn59gc9w3kdklj2lt40000gn/T/ipykernel_70971/3948258616.py:26: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response: AIMessage = model(history)


================================== Ai Message ==================================

Hi Alice!
================================ Human Message =================================

What's the capital of France?
================================== Ai Message ==================================

The capital of France is Paris.
